# Orientation to Python

Chem 6004
January 18, 2019

This will introduce some of the basic concepts required for scientific computing in python.
In particular, the following concepts will be illustrated:


- Basic use of numpy 
- Basic use of matplotlib
- Arrays
- Loops
- Timing
- Functions

We will start with the illustrative example discussed in class, namely, the kinetic energy 
and potential energy of a collection or $N$ charged particle.

\begin{equation}
T = \sum_{l=i}^N \frac{1}{2} m_i v_i^2.
\end{equation}

A natural way to store the masses and velocities of the particles is in an array.  The following lines of code 
will import numpy and create two numpy arrays that can be used to store the masses and velocities of $N=10$ particles


In [2]:

import numpy as np
### Number of particles will be 10
Npart = 10
### create an array 'm' and 'v' to store the masses and velocities of the 10 particles... 
### initially, each entry in 'm' and 'v' will be zero, and we will have to assign values later
m = np.zeros(Npart)
v = np.zeros(Npart)

We can use a for loop to access all the entries in 'm' and 'v' and assign them values.  For simplicity,
we will give each particle the same mass (1.0 in natural units of mass) and the same velocity (2.5 in natural
units of velocity).

In [3]:
for i in range(0,Npart):
    m[i] = 1.0
    v[i] = 2.5

### Now that values have been assigned, print to confirm they are what you expect
print("Printing array of masses: ",m)
print("Printing array of velocities: ",v)

Printing array of masses:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Printing array of velocities:  [2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5]


Finally, we can perform arithmetic operations directly with the arrays to create a new array of kinetic
energies of each particle.  The following line will compute 
\begin{equation}
T_i = \frac{1}{2} m_i v_i^2.
\end{equation}
for each particle indexed by $i$.


In [4]:
T = 1/2 * m * v**2
### confirm that T is indeed an array with an entry for the kinetic energy of each particle
print(T)

[3.125 3.125 3.125 3.125 3.125 3.125 3.125 3.125 3.125 3.125]


We can compute the total kinetic energy by summing up the entries within T.  This can be done using another
for loop, but it can also be done making use of a numpy function called 'sum'.  We will use both to confirm they give 
the same result.

In [5]:
T_tot_loop = 0.
for i in range(0,Npart):
    T_tot_loop = T_tot_loop + T[i]
T_tot_sum = np.sum(T)

print("Result from loop is ",T_tot_loop)
print("Result from numpy sum is ",T_tot_sum)

Result from loop is  31.25
Result from numpy sum is  31.25


Next let's consider the potential energy:
\begin{equation}
V_i = \sum_{j\neqi}^N \frac{q_i q_j}{4\pi \epsilon_0 r_{ij}}
\end{equation}